# Gamma-ray energy deposition

This notebook provides the initial implementation of Gamma-ray energy deposition into an arbitrary ejecta.
It is a WORK IN PROGRESS and should NOT be used for any science work until further notice.

## Main loop

Generates a simple 1D ejecta and a list of gamma-ray objects.

Runs packets of gamma-rays through the ejecta. Handles interactions by calling the appropriate function. 

Adds deposited energy and output energy to 2 different dataframes.

In [1]:
import numpy as np
import astropy as apy

from tardis.energy_input.indivisible_packets import main_gamma_ray_loop

import matplotlib.pyplot as plt
from tardis.model import Radial1DModel
from tardis.io.config_reader import Configuration

Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

In [ ]:
# Set up packet count
num_packets = 50000

# Lock seed
np.random.seed(1)

# Adjust model
config = Configuration.from_yaml("../../tardis/io/tests/data/tardis_configv1_density_exponential_nebular.yml")
config.model.structure.velocity.start = 1 * apy.units.km / apy.units.s
config.model.structure.density.rho_0 = 5e2 * apy.units.g / (apy.units.cm ** 3)
config.supernova.time_explosion = 2.0 * apy.units.d

config.atom_data = "kurucz_cd23_chianti_H_He.h5"

# Create model
model = Radial1DModel.from_config(config)

## Generate plasma

In [ ]:

from tardis.plasma.properties import Density, Abundance, IsotopeAbundance, IsotopeNumberDensity, AtomicData, AtomicMass, IsotopeMass, NumberDensity, SelectedAtoms
from tardis.plasma.base import BasePlasma
from tardis.io.atom_data import AtomData

input = [Density, Abundance, IsotopeAbundance, AtomicData, AtomicMass, IsotopeNumberDensity, NumberDensity, SelectedAtoms, IsotopeMass]

plasma = BasePlasma(
        plasma_properties=input,
        density = model.density,
        abundance = model.abundance,
        isotope_abundance = model.raw_isotope_abundance,
        atomic_data = AtomData.from_hdf(config.atom_data)
    )


In [ ]:

# Compute energy deposition rate
# ejecta_energy_df is the deposited energy
# ejecta_plot_energy_df is information for plotting
# escape_energy is the escaping energy
# decayed_packet_count is the number of packets created per shell
# energy_plot_positrons is the deposited energy from positrons
# estimated_deposition is the deposited energy from the Kasen (2006) estimator (currently not functional)
(
    energy_df,
    energy_plot_df,
    escape_energy,
    decayed_packet_count,
    energy_plot_positrons,
    estimated_deposition
) = main_gamma_ray_loop(
    num_packets,
    model,
    plasma,
    time_steps=50,
    time_end=50.0,
)

ejecta_energy = energy_plot_df["energy_input"]
ejecta_energy_r = energy_plot_df["energy_input_r"]
energy_input_time = energy_plot_df["energy_input_time"]
energy_input_type = energy_plot_df["energy_input_type"]

## Plotting results

Energy deposited at a given radius

In [ ]:
fig = plt.figure(dpi=150, facecolor='w')
ax = fig.add_subplot(111)

scatter = ax.scatter(np.array(ejecta_energy_r)/np.max(model.v_outer.value), np.array(ejecta_energy), s=1, alpha=0.1)
ax.set_xlabel("r/R")
ax.set_ylabel("E (keV)")
ax.semilogy();

## Interactions binned by radius

In [ ]:
fig = plt.figure(dpi=150, facecolor='w')
ax = fig.add_subplot(111)
ax.hist(np.array(ejecta_energy_r), bins=200)
#ax.set_xlim(0, 1)
ax.set_xlabel("r (cm)")
ax.set_ylabel("Interaction count");

## Density Profile

In [ ]:
fig = plt.figure(dpi=150, facecolor='w')
plt.semilogy(model.r_middle/np.max(model.r_outer), model.density)
plt.plot(0,0)
plt.ylabel("Density g cm$^{-3}$")
plt.xlabel("r/R");

## Fraction of energy escaping from the ejecta

In [ ]:
np.sum(np.sum(escape_energy) / (np.sum(escape_energy) + np.sum(energy_df)))

## Spectrum of escape energy at the final time step

In [ ]:
from tardis.energy_input.energy_source import read_artis_lines

ni56_lines = read_artis_lines("ni56")
co56_lines = read_artis_lines("co56")

plt.figure(figsize=(12, 6), dpi=150)
plt.step(escape_energy.index, escape_energy.iloc[:,49], label="$\gamma$-spectrum", where="post")
plt.xlabel("Energy (keV)")
plt.ylabel("keV/keV/s");
plt.loglog()
plt.xlim(100, 4000)
#plt.ylim(0.01, 100)
plt.vlines(ni56_lines.energy*1000, 0.01, 0.02, color="k", label="Ni56")
plt.vlines(co56_lines.energy*1000, 0.01, 0.02, color="r", label="Co56")
plt.legend()

## Energy deposition rate

Dataframe index is the radial grid location. Columns are time steps in seconds

In [ ]:
energy_df

## Energy deposition rate versus radius at time_explosion

In [ ]:
fig = plt.figure(dpi=150, facecolor='w')
plt.semilogy(model.v_middle/np.max(model.v_outer), energy_df.iloc[:, 0])
plt.xlabel("r/R")
plt.ylabel("Energy deposition rate [eV/s$]");